# Density matrices and path integrals


## Quantum Harmonic Oscillator
- Particle of mass $m$ in a potential $V(x) = \frac{1}{2} m \omega^2 x^2$
- Gouverend by Schroedigner Equation:
    - $H\psi = E\psi$
    - $H\psi = \left( - \frac{\hbar^2}{2m} \frac{d^2}{dx^2} + \frac{1}{2} m \omega^2 x^2 \right) \psi$
- Simplify: $\hbar = 1, \qquad m = 1, \qquad \omega = 1$ (this is not a restriction)
- $\Rightarrow H\psi = \left( -\frac{1}{2} \frac{d^2}{dx^2} + \frac{1}{2} x^2 \right) \psi$
    - Time independent
    - $H$ is the Hamilton operator (Hamiltonian $= -\frac{1}{2} \frac{d^2}{dx^2} + \frac{1}{2} x^2$)
- Wavefunctions:
    - $\psi_n(x) \rightarrow 0 \text{ for } x \rightarrow \pm \infty$
    - Normalized: $\int_{-\infty}^{+\infty}dx |\psi_n(x)|^2 = 1$
    - Orthognoal: $\int_{-\infty}^{+\infty}dx \phi_n(x) \phi_m(x) = 0 \text{ for } n \neq m$
- In the code:
    - ``psi_function`` solves the Schroedinger equation ($H\psi_n = E\psi_n$)
    - This can be checked with: $\frac{H\psi_n}{\psi_n} = E = \text{const.}$
    - $\rightarrow$ use discrete approximations for the second derivative


In [ ]:
import math, pylab                                                                                                                   

def psi_function(x, n_states):
    psi = [math.exp(-x ** 2 / 2.0) / math.pi ** 0.25]  # ground state
    psi.append(math.sqrt(2.0) * x * psi[0])         # first excited state
    # other excited states (through recursion):
    for n in range(2, n_states):
        psi.append(math.sqrt(2.0 / n) * x * psi[n - 1] -
                      math.sqrt((n - 1.0) / n) * psi[n - 2])
    return psi


nx = 300  # nx is even, to avoid division by zero
L = 10.0
dx = L / (nx - 1)
x = [- L / 2.0 + i * dx for i in range(nx)]
n_states = 4
#grid_x = [i * 0.1 for i in range(-50, 51)]

# construct wavefunctions:
psi = [[] for n in range(n_states)]
for x_i in x:
    p = psi_function(x_i, n_states)
    for n in range(n_states):
        psi[n].append(p[n])

# TODO orhonomality_check

# local energy check:
H_psi_over_psi = []
for n in range(n_states):
    H_psi = [(- 0.5 * (psi[n][i + 1] - 2.0 * psi[n][i] + psi[n][i - 1]) / dx ** 2 + 0.5 * x[i] ** 2 * psi[n][i]) for i in range(1, nx - 1)] 
    H_psi_over_psi.append([H_psi[i-1] / psi[n][i] for i in range(1, nx - 1)])

# Harmonic oscillator wavefunctions output
for n in range(n_states):
    shifted_psi = [psi[n][i] + n  for i in range(nx)]  # vertical shift
    pylab.plot(x, shifted_psi)
pylab.title('Harmonic oscillator wavefunctions')
pylab.xlabel('$x$', fontsize=16)
pylab.ylabel('$\psi_n(x)$ (shifted)', fontsize=16)
pylab.xlim(-5.0, 5.0)
pylab.show()

import math, pylab 

# Schroedingers equation check output
for n in range(n_states):
    pylab.plot(x[1:-1], [n + 0.5 for i in x[1:-1]], 'k--', lw=1.5)
    pylab.plot(x[1:-1], H_psi_over_psi[n], '-', lw=1.5)
    pylab.xlabel('$x$', fontsize=18)
    pylab.ylabel('$H \psi_%i(x)/\psi_%i(x)$' % (n, n), fontsize=18)
    pylab.xlim(x[0], x[-1])
    pylab.ylim(n, n + 1)
    pylab.title('Schroedinger equation check (local energy)')
    pylab.show()

## Quantum Statistical Mechanics
- Quantum: Particle in state n, the probability to be at x is given by $\psi_n^2$
- Statistical Mechancis: Probaility in state n is $\pi(n) \propto e^{-\frac{E_n}{k_B T}} = e^{-\beta E_n}$
- $\Rightarrow \pi(x, n) \propto e^{-\beta E_N} \cdot | \psi_n(x)^2|$

## Density Matrix, Matrix Squaring
- $\pi(x,n) \propto e^{-\beta E_n}\psi_n(x)\psi_n^*(x)$
- Two different types of probabilities (Boltzmann, Schroedinger)
- !!! Cannot be computed $\Rightarrow$ leads nowhere
- Discard the informatin about the energy level
- Consider diagonal density matrix: $\pi(x) \propto \rho(x,x,\beta) = \sum_n e^{-\beta E_n}\psi_n(x)\psi_n^*(x)$
- Nondiagnoal density matrix: $\rho(x,x',\beta) = \sum_n e^{-\beta E_n}\psi_n(x) \psi_n^*(x')$
- Partition function: $Z(\beta) = \text{Tr} \rho = \int_{-\infty}^{+\infty}dx \rho(x,x,\beta)$
- $= \int_{-\infty}^{+\infty} dx \sum_n e^{-\beta E_n}\psi_n(x)\psi_n^*(x)$
- $= \sum_n e^{-\beta E_n} \int_{-\infty}^{+\infty}dx \psi_n(x) \psi_n^*(x)$
- $= \sum_n e^{-\beta E_n}$
- Properties of Density Matrix:
    - Convoultion: $\int dx' \rho(x,x',\beta_1) \rho(x', x'', \beta_2) = \dots = \rho(x,x'', \beta_1 + \beta_2)$
    - $\Rightarrow$ with $\beta = \frac{1}{T}$ we can halve the temperature if convoluted with itself!!!
    - Free density matrix: $\rho^{\text{free}}(x,x',\beta) = \frac{1}{\sqrt{2\pi\beta}}\exp\left(-\frac{(x-x')^2}{2\beta}\right)$ (explained later)
    - Trotter decomposition $H = H^{\text{free}} + V(x)$ where $H^{\text{free}} = -\frac{1}{2} \frac{\partial^2}{\partial x^2}$, $V(x) = $ potential
    - $\Rightarrow \rho(x,x',\beta) = e^{-\frac{\beta}{2}V(x)} \rho^{\text{free}}(x,x',\beta)e^{-\frac{\beta}{2}V(x')}$, for $\beta \rightarrow 0$
- $\Rightarrow$ from Free density matrix, we obtain the Trotter decompisition. So we obtain the density matrix for small $\beta$ with the convolution we can scale up again to compute any density matrix

In [ ]:
import math, numpy, pylab

# Free off-diagonal density matrix
def rho_free(x, xp, beta):
    return (math.exp(-(x - xp) ** 2 / (2.0 * beta)) /
            math.sqrt(2.0 * math.pi * beta))

# Harmonic density matrix in the Trotter approximation (returns the full matrix)
def rho_harmonic_trotter(grid, beta):
    return numpy.array([[rho_free(x, xp, beta) * numpy.exp(-0.5 * beta * 0.5 * (x ** 2 + xp ** 2))
                         for x in grid] for xp in grid])

x_max = 5.0 
nx = 100 
dx = 2.0 * x_max / (nx - 1)
x = [i * dx for i in range(-(nx - 1) // 2, nx // 2 + 1)] 
beta_tmp = 2.0 ** (-8)                   # initial value of beta (power of 2)
beta     = 2.0 ** 0                      # actual value of beta (power of 2)
rho = rho_harmonic_trotter(x, beta_tmp)  # density matrix at initial beta
while beta_tmp < beta:
    rho = numpy.dot(rho, rho)
    rho *= dx
    beta_tmp *= 2.0 

# graphics output
pylab.imshow(rho, extent=[-x_max, x_max, -x_max, x_max], origin='lower')
pylab.colorbar()
pylab.title('$\\beta = 2^{%i}$' % math.log(beta, 2)) 
pylab.xlabel('$x$', fontsize=18)                                                                                                     
pylab.ylabel('$x\'$', fontsize=18)
pylab.show()

## Feynman Path Integral and Quantum Monte Carlo
- Convolution is nothing else then squaring a matrix
- Squaring a matrix tends to be imposibble analytically
- Squaring a matrix is timeconsuming for large amount of particles $\rightarrow$ numerically not possible
- Feynman Path Integral overcomes this problem
- Instead of evaluating the convolutions
- $\rho(x,x',\beta) = \int dx'' \rho(x,x'',\beta/2) \rho(x'',x', \beta/2)$
- $= \int \int \int dx'' dx''' dx'''' \rho(x,x''',\beta/4) \rho(x''',x'',\beta/4) \rho(x'',x'''',\beta/4) \rho(x'''',x',\beta/4) = \dots$
- $x' = x_0$
- $\Rightarrow \rho(x_0, x_N, \beta) = \int dx_1 dx_2 \dots dx_{N-1} \rho(x_0,x_1,\beta/N) \rho(x_1,x_2,\beta/N) \dots \rho(x_{N-1}, x_N, \beta/N)$
- $Z(\beta) = \text{Tr} \rho = \int dx_0 dx_1 \dots dx_{N-1}  \rho(x_0,x_1,\beta/N) \rho(x_1,x_2,\beta/n) \dots \rho(x_{N-1}, x_0,\beta/N)$
- Variables $x_0$ upto $x_N$ in these integrals is called a path
- $x_k$ is at position $\frac{k\beta}{N}$
- $\Rightarrow$ imaginary variable $\tau$, which goes from $0$ to $\beta$ in little steps: $\Delta \tau = \frac{\beta}{N}$
- $\Rightarrow$ density matrices and partition functions can be expressed as path integrals
- MCMC we can move from one path integral to the next:
    1. Choose $x_k$ ($x_0$, which is between $x_1$ and $x_{N-1}$ can be moved also, which moves the whole path)
    2. Displace $x_k$ by $\delta x$
    3. Compute the weight after and before the move
    4. Accept/reject move with Metropolic acceptance probability
    5. Position $x_k$ is the new sample for a position of a particle

In [ ]:
import math, random, pylab, os      

# TODO understand, make histogram of positions

def rho_free(x, y, beta):        # free off-diagonal density matrix
    return math.exp(-(x - y) ** 2 / (2.0 * beta))

output_dir = 'snapshots_naive_harmonic_path'
if not os.path.exists(output_dir): os.makedirs(output_dir)
def show_path(x, k, x_old, Accepted, step):
    path = x + [x[0]]
    y_axis = range(len(x) + 1)
    if Accepted:
        old_path = x[:]
        old_path[k] = x_old
        old_path = old_path + [old_path[0]]
        pylab.plot(old_path, y_axis, 'ro--', label='old path')
    pylab.plot(path, y_axis, 'bo-', label='new path')
    pylab.legend()
    pylab.xlim(-5.0, 5.0)
    pylab.xlabel('$x$', fontsize=14)
    pylab.ylabel('$\\tau$', fontsize=14)
    pylab.title('Naive path integral Monte Carlo, step %i' % step)
    pylab.show()
    pylab.clf()

beta = 4.0 
N = 8                                                # number of slices
dtau = beta / N 
delta = 1.0                                          # maximum displacement on one slice
n_steps = 30                                         # number of Monte Carlo steps
x = [random.uniform(-1.0, 1.0) for k in range(N)]   # initial path
show_path(x, 0, 0.0, False, 0)
for step in range(n_steps):
    #print 'step',step
    k = random.randint(0, N - 1)                     # randomly choose slice
    knext, kprev = (k + 1) % N, (k - 1) % N          # next/previous slices
    x_old = x[k]
    x_new = x[k] + random.uniform(-delta, delta)     # new position at slice k
    old_weight  = (rho_free(x[knext], x_old, dtau) *
                   rho_free(x_old, x[kprev], dtau) *
                   math.exp(-0.5 * dtau * x_old ** 2)) 
    new_weight  = (rho_free(x[knext], x_new, dtau) *
                   rho_free(x_new, x[kprev], dtau) *
                   math.exp(-0.5 * dtau * x_new ** 2)) 
    if random.uniform(0.0, 1.0) < new_weight / old_weight:
        x[k] = x_new
        Accepted = True
    else:
        Accepted = False
    show_path(x, k, x_old, Accepted, step + 1)